In [1]:
import pandas as pd

In [57]:
fullcsv = pd.read_csv('Full.csv')

In [53]:
fullcsv.columns

Index(['Suburb', 'Address', 'Rooms', 'Type', 'Price', 'Method', 'SellerG',
       'Date', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'CouncilArea', 'Lattitude',
       'Longtitude', 'Regionname', 'Propertycount'],
      dtype='object')

In [58]:
newcsv = fullcsv[['Address', 'Rooms', 'Landsize', 'Type', 'Price']]
newcsv = newcsv[newcsv['Price'] < 110_0000]
newcsv = newcsv[newcsv['Price'] > 100_0000]
newcsv = newcsv.reset_index(drop = True)
newcsv.shape[0]

1224

In [55]:
newcsv.head()

,Address,Rooms,Landsize,Type,Price
0,25 Bloomburg St,2,156.0,h,1035000.0
1,10 Valiant St,2,220.0,h,1097000.0
2,138/56 Nicholson St,3,4290.0,u,1090000.0
3,200 Nicholson St,2,182.0,h,1012500.0
4,27 Charles St,2,172.0,h,1030000.0


In [56]:
import random

info = list()
length = newcsv.shape[0]
if length >= 5:
    for i in range(5):
        idx = random.randint(0, length - 1)
        info.append(list(newcsv.loc[idx]))
elif length > 0:
    for i in range(length):
        idx = random.randint(0, length - 1)
        info.append(list(newcsv.loc[idx]))
    
for row in info:
    if row[3] == 'h' or row[3] == 'H':
        row[3] = 'House'
    elif row[3] == 'u' or row[3] == 'U':
        row[3] = 'Unit'
    elif row[3] == 't' or row[3] == 'T':
        row[3] = 'TownHouse'
    else:
        row[3] = 'Other'

info

[['126 Princes St', 2, 217.0, 'House', 1015000.0],
 ['14 Chusan Ct', 4, 1222.0, 'House', 1063000.0],
 ['1/22 Ogilvie St', 2, nan, 'House', 1077000.0],
 ['3/19 Gordon St', 2, nan, 'Unit', 1050000.0],
 ['19 Ling Ct', 4, 692.0, 'House', 1019500.0]]

In [62]:
#!/usr/bin/python
import sqlite3
import hashlib

class UserDB():
    def __init__(self):
        conn = sqlite3.connect('user.sqlite')
        print("Opened database successfully")
        c = conn.cursor()
        c.execute('''CREATE TABLE  IF NOT EXISTS UserInfo
               (ID INT PRIMARY KEY     NOT NULL,
               USERNAME           TEXT    NOT NULL,
               PASSWORD           TEXT     NOT NULL);''')
        print("Table created successfully")
        
        res = c.execute("SELECT COUNT(*) FROM UserInfo")
        info = list()
        for row in res:
            info.append(row)
        self.ID = info[0][0]
        print(self.ID)
        conn.commit()
        conn.close()

    def insert(self, User, Password):
        conn = sqlite3.connect('user.sqlite')
        c = conn.cursor()
        self.ID += 1
        cursor = c.execute("SELECT USERNAME, PASSWORD from UserInfo WHERE \
        USERNAME = ?", (User,))

        pwd = hashlib.md5()
        pwd.update(Password.encode('utf-8'))
        pwd = pwd.hexdigest()

        info = list()
        for row in cursor:
            info.append(row)
        if len(info) == 0: # insert
            c.execute("INSERT INTO UserInfo (ID, USERNAME, PASSWORD) \
              VALUES (?, ?, ?)", (self.ID, User, pwd));
        else:
            c.execute("UPDATE UserInfo SET PASSWORD = ? WHERE USERNAME = ?",
                      (pwd, User));
        conn.commit()
        conn.close()

    def check(self, User, Password):
        conn = sqlite3.connect('user.sqlite')
        c = conn.cursor()
        cursor = c.execute("SELECT USERNAME, PASSWORD from UserInfo WHERE \
        USERNAME = ?", (User,))
        info = list()
        for row in cursor:
            info.append(row)

        if len(info) == 0:
            conn.commit()
            conn.close()
            return False

        pwd = hashlib.md5()
        pwd.update(Password.encode('utf-8'))
        pwd = pwd.hexdigest()
        if info[0][1] != pwd:
            conn.commit()
            conn.close()
            return False

        conn.commit()
        conn.close()
        return True

if __name__ == '__main__':
    userdb = UserDB()
    userdb.insert('Xavier', '12345678')
    userdb.insert('Eric', '12345678')
    userdb.check('Patrick', '12345678')
    userdb.insert('Patrick', '12345678')
    userdb.check('Patrick', '12345678')
    userdb.check('Patrick', 'abcdefg')

    print('new password')
    userdb.insert('Patrick', 'abcdefg')
    userdb.check('Patrick', '12345678')
    userdb.check('Patrick', 'abcdefg')


Opened database successfully
Table created successfully
2


False

True

False

new password


False

True